<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: Data Wrangling II
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Tópicos**

<ol type="1">
  <li>Agregação e Ordenação;</li>
  <li>Combinação;</li>
  <li>Técnicas Avançadas.</li>
</ol>

---

# **Exercícios**

Neste exercícios, vamos trabalhar com dados geográficos, demográficos e econômicos do Brasil. Vamos manipular e combinar dados de duas frentes distintas para poder responder perguntas de negócios.

## 1\. Data Wrangling

### **1.1. Estados**

O arquivo `estados-bruto.xml` contém informações sobre estados (nome, sigla e região). **Carregue-o na máquina virtual do Google Colab**. Um arquivo do tipo XML é similar a um arquivo do tipo HTML, exemplo do estado do Acre:

```xml
<ESTADO>
  <ID>1</ID>
  <NOME>ACRE</NOME>
  <IDCAPITAL>16</IDCAPITAL>
  <SIGLA>AC</SIGLA>
  <REGIAO>NORTE</REGIAO>
</ESTADO>
```

Utilize o pacote Python `beautifulsoup4` para extrair os dados do arquivo `estados-bruto.xml` providenciado. Salve os dados extraidos no arquivo `estados-limpo.csv` separado por `;`. Exemplo das três primeiras linhas mais o cabeçalho:

```
estado;sigla;regiao
ACRE;AC;NORTE
ALAGOAS;AL;NORDESTE
AMAPA;AP;NORTE
```

**Dica**: Utilize o parser de xml chamado `lxml` do `beautifulsoup4`.

In [ ]:
import csv
from bs4 import BeautifulSoup


with open('estados-bruto.xml', 'r', encoding='utf-8') as file:
    content = file.read()

soup = BeautifulSoup(content, 'lxml')

estados = soup.find_all('estado')

data = []
for estado in estados:
    nome = estado.find('nome').text
    sigla = estado.find('sigla').text
    regiao = estado.find('regiao').text
    data.append({'estado': nome, 'sigla': sigla, 'regiao': regiao})

csv_filename = 'estados-limpo.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['estado', 'sigla', 'regiao']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames, delimiter=';')
    writer.writeheader()
    writer.writerows(data)

print(estados)
print(f'Dados salvos em {csv_filename}')


### **1.2. Cidades**

O arquivo `cidades-bruto.csv` contém informações demográficas e socioeconomicas das cidades do Brasil. **Carregue-o na máquina virtual do Google Colab**. Utilize o pacote Python `pandas` para extrair os dados do arquivo `cidades-bruto.xml` providenciado. Seguindo as seguintes especificações:

1. Apenas dados do censo de 2010;
1. Apenas as colunas UF, Nome, PIB, Pop_est_2009 e PIB_percapita.

Salve os dados extraidos no arquivo `cidades-limpo.csv` separado por `;`. Exemplo das três primeiras linhas mais o cabeçalho:

```
estado;cidade;populacao;pib;pib_percapita
BAHIA;TREMEDAL;18433;57883.9921875;3140.23999023
RIO GRANDE DO SUL;TURUÇU;4000;45723875;11430.96972656
ESPIRITO SANTO;VITÓRIA;320156;19782628;61790.58984375
```

In [214]:
import pandas as pd

with open('cidades-bruto.csv', 'r', encoding='utf-8') as file:
    content = file.read()

cidades = pd.read_csv('cidades-bruto.csv', usecols=['UF', 'nome', 'Pop_est_2009', 'PIB', 'PIB_percapita'])
cidades.head()

cidades.columns = ['estado', 'cidade', 'pib', 'populacao', 'pib_percapita']

csv_filename = 'cidades-limpo.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
   cidades.to_csv(csv_filename, sep=';', index=False, encoding='utf-8')



### **1.3. Brasil**

Utilize o pacote Python `pandas` para combinar os dados do arquivo `estados-limpo.csv` com os dados do arquivo `cidades-limpo.csv` em um único dataframe. Escolha a coluna e o método de combinação de tal forma que **não haja perda de dados** no processo (não produzirá valores nulos `NaN`). Salve os dados do dataframe no arquivo `brasil.csv`

In [ ]:
import pandas as pd

brasil_df = pd.read_csv('estados-limpo.csv', delimiter=';')
brasil_df = pd.read_csv('cidades-limpo.csv', delimiter=';')

csv_filename = 'brasil.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    brasil_df.to_csv(csv_filename, sep=';', index=False, encoding='utf-8')

    print('Dados salvos em brasil.csv')
    print(brasil_df)


---

## 2\. Data Analytics

### **2.1. DataFrame**

Utilize o pacote Python `pandas` para carregar o arquivo `brasil.csv` no dataframe `brasil_df`.

In [ ]:
brasil_df = pd.read_csv('brasil.csv', delimiter=';')

print(brasil_df)

### **2.2. Analise**

Utilize o dataframe `brasil_df` para responder as seguintes perguntas de negócio:

 - Quais são as 10 cidades mais populosas do Brasil?

In [ ]:
top10_cidades_populosas = brasil_df.nlargest(10, 'populacao')[['cidade', 'populacao']]

print(top10_cidades_populosas)

 - Quais são as 5 cidades com a menor PIB per capita da região nordeste?

In [ ]:
menor_pib_nordeste = brasil_df.nsmallest(5, 'pib_percapita')[['cidade', 'estado']]

print(menor_pib_nordeste)

 - Quais são as 15 cidades com maior PIB do do estado de São Paulo?

In [ ]:
cidades_maior_pib_sp = brasil_df[brasil_df['estado'] == 'SÃO PAULO'].nlargest(15, 'pib')[['cidade', 'pib']]

print(cidades_maior_pib_sp)


 - Qual é o PIB do estado de Santa Catarina?

In [ ]:
pib_santa_catarina = brasil_df[brasil_df['estado'] == 'SANTA CATARINA']['pib'].sum()

print(pib_santa_catarina)

 - Qual é o população da região sul?

In [ ]:
brasil_df.groupby(['estado', 'cidade']). sum()

In [ ]:
populacao_regiao_sul = brasil_df[brasil_df['estado'] == 'PARANÁ']['populacao'].sum()

print(populacao_regiao_sul)

 - Qual é o PIB per capito médio das cidades do Mato Grosso do Sul?

In [ ]:
pib_percapita_medio_ms = brasil_df[brasil_df['estado'] == 'MATO GROSSO DO SUL']['pib_percapita'].mean()

print(f"O PIB per capita médio das cidades do Mato Grosso do Sul é: {pib_percapita_medio_ms}")

 - Qual é a população do Brasil?

In [ ]:
populacao_brasil = brasil_df['populacao'].sum()
print(f"A população do Brasil é: {populacao_brasil}")

### **2.3. Visualização**

Utilize o dataframe `brasil_df` para gerar as seguintes visualizações.

 - Gere um gráfico de barras com as 10 cidades menos populosas do Brasil.

In [ ]:
import matplotlib.pyplot as plt

# Supondo que a coluna de população no DataFrame seja chamada 'populacao'
top10_menos_populosas = brasil_df.nsmallest(10, 'populacao')[['cidade', 'populacao']]

# Criar o gráfico de barras
plt.figure(figsize=(10, 6))
plt.barh(top10_menos_populosas['cidade'], top10_menos_populosas['populacao'], color='skyblue')
plt.xlabel('População')
plt.title('As 10 Cidades Menos Populosas do Brasil')
plt.gca().invert_yaxis()  # Inverter a ordem para a cidade menos populosa estar no topo
plt.show()


 - Gere um gráfico de pizza com a proporção da população do Brasil por região.

In [ ]:
import matplotlib.pyplot as plt

# Agrupe por região e calcule a população total de cada região
populacao_por_regiao = brasil_df.groupby('estado')['populacao'].sum()

# Criar o gráfico de pizza
plt.figure(figsize=(8, 8))
plt.pie(populacao_por_regiao, labels=populacao_por_regiao.index, autopct='%1.1f%%', startangle=140, colors=['lightcoral', 'lightskyblue', 'lightgreen', 'gold'])
plt.title('Proporção da População do Brasil por Região')
plt.show()
